In [ ]:
!pip install numpy==1.26.4
!pip install scikit-surprise --upgrade --quiet
import numpy as np
print(np.__version__)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
1.26.4


In [ ]:
!pip install scikit-surprise --quiet


#Import Libraries

In [ ]:

from surprise import SVD, Dataset, Reader, accuracy
from surprise.model_selection import train_test_split

In [ ]:
import pandas as pd
import numpy as np

#load the data

In [ ]:
from google.colab import files
uploaded = files.upload()


Saving ratings_10k.csv to ratings_10k (1).csv


In [ ]:
import pandas as pd

df = pd.read_csv("ratings_10k.csv")


In [ ]:
df.head()

,userId,movieId,rating
0,103,182,5
1,436,13,4
2,349,285,4
3,271,259,1
4,107,90,1


#preprocessing
convert the dataframe into surprise dataset



In [ ]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df[['userId', 'movieId', 'rating']], reader)


"The code imports necessary libraries, defines the rating scale (1–5), and prepares the dataset (into DataFrame) in a format that Surprise can use for building recommendation models".

#Train-Test split

In [ ]:

trainset, testset = train_test_split(data, test_size=0.2)
trainset, testset

(<surprise.trainset.Trainset at 0x7e20406cdd30>,
 [(258, 106, 4.0),
  (442, 41, 4.0),
  (73, 94, 2.0),
  (147, 260, 1.0),
  (230, 112, 2.0),
  (237, 192, 1.0),
  (405, 137, 5.0),
  (491, 201, 1.0),
  (376, 98, 3.0),
  (124, 158, 3.0),
  (308, 180, 5.0),
  (405, 224, 1.0),
  (137, 189, 3.0),
  (95, 254, 2.0),
  (16, 129, 4.0),
  (233, 156, 3.0),
  (378, 30, 5.0),
  (78, 137, 1.0),
  (92, 14, 3.0),
  (481, 204, 5.0),
  (136, 264, 5.0),
  (192, 236, 1.0),
  (36, 38, 5.0),
  (324, 89, 3.0),
  (168, 52, 5.0),
  (44, 59, 2.0),
  (7, 5, 4.0),
  (141, 244, 3.0),
  (114, 131, 3.0),
  (183, 171, 2.0),
  (144, 26, 5.0),
  (331, 86, 1.0),
  (32, 235, 3.0),
  (190, 178, 4.0),
  (388, 237, 1.0),
  (106, 239, 2.0),
  (289, 184, 4.0),
  (440, 268, 4.0),
  (140, 112, 4.0),
  (412, 288, 1.0),
  (32, 273, 5.0),
  (468, 80, 1.0),
  (391, 241, 1.0),
  (392, 187, 2.0),
  (354, 71, 2.0),
  (461, 246, 2.0),
  (210, 109, 5.0),
  (243, 204, 1.0),
  (215, 150, 1.0),
  (282, 241, 5.0),
  (140, 291, 4.0),
  (285, 

"The dataset was successfully divided into a training set and a testing set. The training set was converted into Surprise’s internal format for model learning, while the testing set was kept as userID–movieID–rating tuples to evaluate predictions. This ensures the model can be trained effectively and its accuracy can be measured on unseen data"

#Build Model (SVD)

In [ ]:
from surprise import Dataset, Reader, SVD, accuracy
from surprise.model_selection import train_test_split
model = SVD()
model.fit(trainset)



"SVD learns hidden patterns from the userId–movieId–rating matrix and predicts ratings for user–movie pairs that are not yet rated"

#Test Predictions

In [ ]:

predictions = model.test(testset)

print(model.predict(uid=1, iid=50))


user: 1          item: 50         r_ui = None   est = 3.39   {'was_impossible': False}


"The model predicts that user 1 would rate movie 50 with approximately 3.24 stars out of 5. Since the true rating is unknown, this estimated score can be used for making recommendations"

#Evaluate performance

In [ ]:
# Calculate RMSE and MAE
rmse = accuracy.rmse(predictions)
mae = accuracy.mae(predictions)

print("Model Performance:")
print("RMSE:", rmse)
print("MAE:", mae)


RMSE: 1.4729
MAE:  1.2723
Model Performance:
RMSE: 1.4728770974301473
MAE: 1.27227189618332


"The SVD model was evaluated using RMSE and MAE. Results show that, on average, the model’s predictions differ from the actual ratings by about 1.27 to 1.47 stars. This indicates that while the model captures some user–movie rating patterns, its accuracy could be improved with further tuning or additional features"

In [ ]:
for uid, iid, true_r, est, _ in predictions[:10]:
    print(f"User {uid} rated Movie {iid} {true_r}, model predicted {round(est,2)}")


User 258 rated Movie 106 4.0, model predicted 2.9
User 442 rated Movie 41 4.0, model predicted 4.17
User 73 rated Movie 94 2.0, model predicted 2.97
User 147 rated Movie 260 1.0, model predicted 2.94
User 230 rated Movie 112 2.0, model predicted 3.16
User 237 rated Movie 192 1.0, model predicted 3.99
User 405 rated Movie 137 5.0, model predicted 3.27
User 491 rated Movie 201 1.0, model predicted 2.57
User 376 rated Movie 98 3.0, model predicted 3.15
User 124 rated Movie 158 3.0, model predicted 3.28


"The comparison between actual ratings and predicted ratings shows that the SVD model captures general rating patterns but often deviates by 1–2 stars for individual cases. For example, some low ratings (1.0 or 2.0) were overestimated around 3.0, while higher ratings (such as 5.0) were underestimated. This indicates that while the model provides reasonable predictions on average, its accuracy for individual users and movies can vary significantly"